## Загрузка библиотек

In [462]:
import tarfile
import re

from sklearn.model_selection import train_test_split

from collections import Counter, defaultdict
import random

## 1: Препроцессинг

Загружаем датасет [French-English, 292 short sentences](https://www.cis.lmu.de/~fraser/EMA2008/model1.html)
(для этого надо нажать на "German-English, 292 short sentences", т.к. на сайте перепутаны ссылки на скачивание)

In [463]:
#извлекаем файлы из архива
with tarfile.open('fr-en.tiny.tgz', 'r:gz') as tar:
  tar.extractall()

Вызовем команду shell `ls`, чтобы увидеть, что мы распаковали.

In [464]:
!ls

fr-en.tiny.en   fr-en.tiny.fr   fr-en.tiny.tgz  ДЗ3_smt.ipynb


Создаем 2 выборки и токенизируем тексты по фразам.

In [466]:
with open('fr-en.tiny.en', 'r') as f:
  english = f.read().split('\n')[:-1]

with open('fr-en.tiny.fr', 'r') as f:
  french = f.read().split('\n')[:-1]

def cleaning_(lang):
    #топорный метод, но в "белый лист" добавляем только буквы
    whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZàâäçèéêëîïôœùûüÿÀÂÄÇÈÉÊËÎÏÔŒÙÛÜŸ')
    result_list = []
    for sentence in lang:
        #приводим всё к нижнему регистру, оставляем только буквы из "белого листа"
        result_list.append((''.join(filter(whitelist.__contains__, sentence))).lower())
    return result_list

english = cleaning_(english)
french = cleaning_(french)

print("Данные языка X:\n", english[:7]) #выведем 7 предложений
print("Данные языка Y:\n", french[:7])

Данные языка X:
 ['resumption of the session', 'please rise  then  for this minute  s silence ', ' the house rose and observed a minute  s silence ', 'madam president  on a point of order ', 'if the house agrees  i shall do as mr evans has suggested ', 'madam president  on a point of order ', 'i would like your advice about rule  concerning inadmissibility ']
Данные языка Y:
 ['reprise de la session', 'je vous invite à vous lever pour cette minute de silence ', ' le parlement  debout  observe une minute de silence ', 'madame la présidente  c est une motion de procédure ', 'si l assemblée en est d accord  je ferai comme m evans l a suggéré ', 'madame la présidente  c est une motion de procédure ', 'je voudrais vous demander un conseil au sujet de l article   qui concerne l irrecevabilité ']


Разделим нашу выборку с помощью sklearn.

In [467]:
X_train, X_test, y_train, y_test = train_test_split(english, french)

cn = 0
print("> Обучающая выборка:")
for text, label in zip(X_train, y_train):
    print(f"\nТекст на французском: {label}\n Его перевод на английский: {text}\n")
    cn+=1
    if cn==3:
        break #чтобы выводились не все предложения, а только часть

cn = 0
print("> Тестовая выборка:")
for text, label in zip(X_test, y_test):
    print(f"\nТекст на французском: {label}\n Его перевод на английский: {text}\n")
    cn+=1
    if cn==3:
        break

> Обучающая выборка:

Текст на французском: merci  monsieur segni  je le ferai bien volontiers 
 Его перевод на английский: thank you  mr segni  i shall do so gladly 


Текст на французском: et pareil phénomène ne devrait pas occuper nos débats 
 Его перевод на английский: should we not be discussing this issue 


Текст на французском: elle devrait poursuivre dans cette voie 
 Его перевод на английский: it should continue along this path 

> Тестовая выборка:

Текст на французском: je vous demande donc à nouveau de faire le nécessaire pour que nous puissions disposer d une chaîne néerlandaise 
 Его перевод на английский: i would therefore once more ask you to ensure that we get a dutch channel as well 


Текст на французском: j ai voté  pour  
 Его перевод на английский: my vote was  in favour  


Текст на французском: je pense qu il est aussi nécessaire de relever le règlement minimal 
 Его перевод на английский: i believe there is also a need to raise the de minimis regulation 



### Подготовка данных

Проведем пословную токенизацию каждой фразы.

In [470]:
def tokenize(sentences):
  #функция возвращает списки слов
  return [sentence.split() for sentence in sentences]

X_train_tokens, X_test_tokens, y_train_tokens, y_test_tokens = tokenize(X_train), tokenize(X_test), tokenize(y_train), tokenize(y_test)

print('Образец токенизированного текста:', X_train_tokens[:10])

Образец токенизированного текста: [['thank', 'you', 'mr', 'segni', 'i', 'shall', 'do', 'so', 'gladly'], ['should', 'we', 'not', 'be', 'discussing', 'this', 'issue'], ['it', 'should', 'continue', 'along', 'this', 'path'], ['i', 'would', 'like', 'to', 'mention', 'one', 'final', 'point'], ['i', 'thought', 'that', 'it', 'was', 'quite', 'superb'], ['when', 'will', 'we', 'debate', 'that'], ['i', 'hope', 'that', 'the', 'commission', 'is', 'able', 'to', 'accept', 'the', 'present', 'amendment'], ['how', 'do', 'we', 'ensure', 'that', 'unlawful', 'state', 'aid', 'is', 'paid', 'back'], ['this', 'is', 'the', 'way', 'forward', 'if', 'we', 'are', 'to', 'ensure', 'uniform', 'conditions', 'of', 'competition'], ['in', 'order', 'to', 'function', 'the', 'market', 'needs', 'rules']]


Создадим словарь уникальных словоформ.

In [471]:
x_vocab = Counter(' '.join(french).split()).keys()
y_vocab = Counter(' '.join(english).split()).keys()

print(f"Словарь французских словоформ: {list(x_vocab)[:50]}...\n Всего {len(x_vocab)} словоформ")
print(f"\nCловарь английских словоформ: {list(y_vocab)[:50]}...\n Всего {len(y_vocab)} словоформ")

Словарь французских словоформ: ['reprise', 'de', 'la', 'session', 'je', 'vous', 'invite', 'à', 'lever', 'pour', 'cette', 'minute', 'silence', 'le', 'parlement', 'debout', 'observe', 'une', 'madame', 'présidente', 'c', 'est', 'motion', 'procédure', 'si', 'l', 'assemblée', 'en', 'd', 'accord', 'ferai', 'comme', 'm', 'evans', 'a', 'suggéré', 'voudrais', 'demander', 'un', 'conseil', 'au', 'sujet', 'article', 'qui', 'concerne', 'irrecevabilité', 'il', 'précise', 'que', 'cela']...
 Всего 1081 словоформ

Cловарь английских словоформ: ['resumption', 'of', 'the', 'session', 'please', 'rise', 'then', 'for', 'this', 'minute', 's', 'silence', 'house', 'rose', 'and', 'observed', 'a', 'madam', 'president', 'on', 'point', 'order', 'if', 'agrees', 'i', 'shall', 'do', 'as', 'mr', 'evans', 'has', 'suggested', 'would', 'like', 'your', 'advice', 'about', 'rule', 'concerning', 'inadmissibility', 'it', 'says', 'that', 'should', 'be', 'done', 'despite', 'principle', 'relative', 'stability']...
 Всего 978 сло

## 2: Модель SMT

### IBM 1 Expectation-Maximization (t-model)

In [472]:
#вероятность того, что случайное слово x_vocab соответсвует случайному слову y_vocab
uniform = 1 / (len(x_vocab) * len(y_vocab))

f'{uniform:.10f}'

'0.0000009459'

In [473]:
#t-model
t = {}

for i in range(len(X_train)):
  #начинаем итерацию по обучающей выборке
  for word_x in X_train_tokens[i]:
    for word_y in y_train_tokens[i]:
      #создаем t-table
      t[(word_x, word_y)] = uniform

#t-table
for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Соответствие | thank   ->   merci | Вероятность: 0.0
Соответствие | thank   ->   monsieur | Вероятность: 0.0
Соответствие | thank   ->   segni | Вероятность: 0.0
Соответствие | thank   ->   je | Вероятность: 0.0
Соответствие | thank   ->   le | Вероятность: 0.0
Соответствие | thank   ->   ferai | Вероятность: 0.0
Соответствие | thank   ->   bien | Вероятность: 0.0
Соответствие | thank   ->   volontiers | Вероятность: 0.0
Соответствие | you   ->   merci | Вероятность: 0.0
Соответствие | you   ->   monsieur | Вероятность: 0.0
Соответствие | you   ->   segni | Вероятность: 0.0
Соответствие | you   ->   je | Вероятность: 0.0
Соответствие | you   ->   le | Вероятность: 0.0
Соответствие | you   ->   ferai | Вероятность: 0.0
Соответствие | you   ->   bien | Вероятность: 0.0
Соответствие | you   ->   volontiers | Вероятность: 0.0
Соответствие | mr   ->   merci | Вероятность: 0.0
Соответствие | mr   ->   monsieur | Вероятность: 0.0
Соответствие | mr   ->   segni | Вероятность: 0.0
Соответствие 

In [474]:
#количество итераций обучения
epochs = 14

In [475]:
for epoch in range(epochs):
  #начинаем обучение

  #шаг 0. создаем слоты для подсчета статистики
  count = {} # P(x|y)
  total = {} # P(y)

  for i in range(len(X_train)):
    #начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        #создаем слоты для подсчета условной вероятности совпадений в корпусе
        count[(word_x, word_y)] = 0
        #и слоты для статистической языковой модели y
        total[word_y] = 0

  #шаг 1. Expectation
  for i in range(len(X_train)):
    #начинаем итерацию по обучающей выборке
    total_stat = {} # статистика x

    #собираем предварительную статистику на основе данных x
    for word_x in X_train_tokens[i]:
      total_stat[word_x] = 0 # создаем слоты для подсчета статистики по каждому токену x
      for word_y in y_train_tokens[i]:
        #обновляем данные из t-table; увеличиваем значения при обнаружении совместной встречаемости
        total_stat[word_x] += t[(word_x, word_y)]

    #обновляем данные для P(x|y) и P(y)
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        #подсчет условной вероятности совпадений в корпусе: равномерное распределение / частотность x
        count[(word_x, word_y)] += t[(word_x, word_y)] / total_stat[word_x]
        #подсчет статистической информации y: равномерное распределение / частотность x
        total[word_y] += t[(word_x, word_y)] / total_stat[word_x]

  #шаг 2. Maximization
  for i in range(len(X_train)):
    #начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        #обновляем t-table: вероятность совпадения в корпусе / вероятность информации y
        t[(word_x, word_y)] = count[(word_x, word_y)] / total[word_y]

for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Соответствие | thank   ->   merci | Вероятность: 0.372
Соответствие | thank   ->   monsieur | Вероятность: 0.001
Соответствие | thank   ->   segni | Вероятность: 0.087
Соответствие | thank   ->   je | Вероятность: 0.0
Соответствие | thank   ->   le | Вероятность: 0.0
Соответствие | thank   ->   ferai | Вероятность: 0.0
Соответствие | thank   ->   bien | Вероятность: 0.0
Соответствие | thank   ->   volontiers | Вероятность: 0.087
Соответствие | you   ->   merci | Вероятность: 0.339
Соответствие | you   ->   monsieur | Вероятность: 0.082
Соответствие | you   ->   segni | Вероятность: 0.059
Соответствие | you   ->   je | Вероятность: 0.0
Соответствие | you   ->   le | Вероятность: 0.0
Соответствие | you   ->   ferai | Вероятность: 0.0
Соответствие | you   ->   bien | Вероятность: 0.0
Соответствие | you   ->   volontiers | Вероятность: 0.059
Соответствие | mr   ->   merci | Вероятность: 0.0
Соответствие | mr   ->   monsieur | Вероятность: 0.915
Соответствие | mr   ->   segni | Вероятность:

Получается правдоподобно, например:

these   ->   ces | Вероятность: 0.975

the   ->   le | Вероятность: 0.998

i   ->   je | Вероятность: 0.979

especially   ->   particulièrement | Вероятность: 0.961

creation   ->   création | Вероятность: 0.926

commission   ->   commission | Вероятность: 0.954

not   ->   pas | Вероятность: 0.997

### Биграммная модель

In [477]:
#для обучения модели объединим 2 выборки
tokens = ' '.join(french).split()

#хранилище для биграмм
bigram_model = defaultdict(list)

#собираем все попарные совпадения
for i in range(len(tokens)-1):
    current_word = tokens[i]
    next_word = tokens[i + 1]
    bigram_model[current_word].append(next_word)

print(list(bigram_model)[:10])

def decoder(model, steps=5):
  #инициализация случайного токена
  current_word = random.choice(tokens)
  generated_sentence = current_word

  for step in range(steps):
    #пошаговая генерация
    print('Шаг', step+1)
    next_word_options = model[current_word]
    print(f'Правдоподобные варианты продолжения для токена {current_word}:', next_word_options)

    current_word = random.choice(next_word_options)
    generated_sentence += ' '
    generated_sentence += current_word
    print('Промежуточный результат:', generated_sentence)
    print()
  print('Результат:', generated_sentence)

decoder(bigram_model)

['reprise', 'de', 'la', 'session', 'je', 'vous', 'invite', 'à', 'lever', 'pour']
Шаг 1
Правдоподобные варианты продолжения для токена régir: ['ces']
Промежуточный результат: régir ces

Шаг 2
Правдоподобные варианты продолжения для токена ces: ['risques', 'transports', 'fonds', 'aspects', 'modifications', 'orientations', 'mémorandums', 'monopoles', 'moyens']
Промежуточный результат: régir ces fonds

Шаг 3
Правдоподобные варианты продолжения для токена fonds: ['structurels', 'de', 'c', 'sont', 'structurels', 'structurels', 'structurels', 'de', 'j', 'structurels']
Промежуточный результат: régir ces fonds structurels

Шаг 4
Правдоподобные варианты продолжения для токена structurels: ['fonds', 'ont', 'le', 'et', 'de']
Промежуточный результат: régir ces fonds structurels de

Шаг 5
Правдоподобные варианты продолжения для токена de: ['la', 'silence', 'silence', 'procédure', 'procédure', 'l', 'stabilité', 'faire', 'santé', 'consignes', 'prolonger', 'coup', 'l', 'personnes', 'lundi', 'modificati

## 3: Оценка результатов

In [478]:
#сортировка t-table по убыванию правдоподобия
sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)
#print(sorted_t)
def translate(token):
  for element in sorted_t:
    if element[0][1] == token:
      # поиск совпадений в t-table
      return element[0][0]
        
for sentence in y_test_tokens:
  print("Оригинальное предложение:", ' '.join(sentence))
  translation = []
  for token in sentence:
    if translate(token):
        translation.append(translate(token))
  print("Перевод:", ' '.join(translation))

Оригинальное предложение: je vous demande donc à nouveau de faire le nécessaire pour que nous puissions disposer d une chaîne néerlandaise
Перевод: i you this therefore to of to the necessary to that we of a
Оригинальное предложение: j ai voté pour
Перевод: i i to
Оригинальное предложение: je pense qu il est aussi nécessaire de relever le règlement minimal
Перевод: i too than it is have necessary of the regulation
Оригинальное предложение: par conséquent la commission devra enfin examiner cette question particulière
Перевод: by therefore the commission method this prerequisites
Оригинальное предложение: il s agit de l harmonisation des exigences des examens mais également des exigences minimales
Перевод: it is viability of the to those to that also to those minimum
Оригинальное предложение: en outre les états membres doivent notifier par avance à la commission leurs projets d octroi d aides
Перевод: in the states member must by to the commission workers projects of of aid
Оригинальное 

In [479]:
from nltk.translate.bleu_score import corpus_bleu

print(y_test_tokens[0])
candidate = [translate(token) for token in y_test_tokens[0] if translate(token)]
reference = [X_test_tokens[i] for i in range(len(candidate))]

print('reference:    ',reference)
print('candidate:     ',candidate)
print(len(reference), len(candidate))
bleu_score = corpus_bleu(reference, candidate)

print("BLEU Score:", bleu_score)

['je', 'vous', 'demande', 'donc', 'à', 'nouveau', 'de', 'faire', 'le', 'nécessaire', 'pour', 'que', 'nous', 'puissions', 'disposer', 'd', 'une', 'chaîne', 'néerlandaise']
reference:     [['i', 'would', 'therefore', 'once', 'more', 'ask', 'you', 'to', 'ensure', 'that', 'we', 'get', 'a', 'dutch', 'channel', 'as', 'well'], ['my', 'vote', 'was', 'in', 'favour'], ['i', 'believe', 'there', 'is', 'also', 'a', 'need', 'to', 'raise', 'the', 'de', 'minimis', 'regulation'], ['therefore', 'the', 'commission', 'should', 'address', 'the', 'issue', 'once', 'and', 'for', 'all'], ['we', 'are', 'concerned', 'here', 'with', 'the', 'harmonisation', 'of', 'examination', 'requirements', 'but', 'also', 'in', 'fact', 'with', 'minimum', 'requirements'], ['furthermore', 'the', 'member', 'states', 'must', 'give', 'the', 'commission', 'advance', 'warning', 'with', 'regard', 'to', 'their', 'intentions', 'concerning', 'state', 'aid'], ['i', 'should', 'like', 'to', 'mention', 'a', 'few', 'specific', 'points'], ['the